In [1]:
import os, sys
%pwd

'/config/workspace/notebook'

In [2]:
os.chdir("../")
%pwd

'/config/workspace'

In [3]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class ModelTrainingConfig:
    root_dir: Path 
    train_data_path: Path 
    test_data_path: Path 
    model_name: str 
    n_estimators: int 
    max_depth: int 
    criterion: str
    target_column: str 

In [4]:
from income_inequality.constants import * 
from income_inequality.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_trainer_config(self) -> ModelTrainingConfig:
        config = self.config.model_trainer
        params = self.params.ExtraTreesClassifier
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

    
        model_trainer_config = ModelTrainingConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            criterion = params.criterion,
            max_depth = params.max_depth,
            target_column = schema.name 
        )

        return model_trainer_config

In [6]:
import pandas as pd
import os 
from income_inequality.logging import logger 
from sklearn.ensemble import ExtraTreesClassifier
import joblib

In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig):
        self.config = config 

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        X_train = train_data.drop([self.config.target_column], axis=1)
        X_test = test_data.drop([self.config.target_column], axis=1)
        y_train = train_data[[self.config.target_column]]
        y_test = test_data[[self.config.target_column]]


        extree = ExtraTreesClassifier(n_estimators=self.config.n_estimators, max_depth=self.config.max_depth,
                                            criterion=self.config.criterion)
        extree.fit(X_train, y_train)

        joblib.dump(extree, os.path.join(self.config.root_dir, self.config.model_name))

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e 

[{asctime}s: INFO: common: yaml file: config/config.yaml loaded successfully]
[{asctime}s: INFO: common: yaml file: params.yaml loaded successfully]
[{asctime}s: INFO: common: yaml file: schema.yaml loaded successfully]
[{asctime}s: INFO: common: created directory at: artifacts]
[{asctime}s: INFO: common: created directory at: artifacts/model_trainer]


/tmp/ipykernel_3037/1009470376.py:17: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  extree.fit(X_train, y_train)
